In [2]:
import pandas as pd

# import the data
df_Wind = pd.read_csv('helio4cast_icmecat.csv')

# don't need to do this as it is already in the correct format
# df_Wind['sc_time'] = pd.to_datetime(df_Wind['sc_time'])

In [3]:
# create a 3d plot template
import plotly.express as px
import plotly.graph_objects as go
import numpy as np

def GSE_sc_plot(plot_df, plot_title):

    # Convert datetime to float for linear scaling (days since epoch)
    event_time_dt = pd.DatetimeIndex(plot_df['sc_time']) #------------------------------
    event_time_numeric = (event_time_dt - pd.Timestamp("1970-01-01")) / pd.Timedelta(days=1)

    # Also create string labels for hover
    event_dates_str = event_time_dt.strftime('%Y-%m-%d')

    # Spherical to Cartesian
    x = plot_df['mo_sc_GSE_x'] #------------------------------
    y = plot_df['mo_sc_GSE_y'] #------------------------------
    z = plot_df['mo_sc_GSE_z'] #------------------------------

    ## Create DataFrame
    df = pd.DataFrame({
        'x': x,
        'y': y,
        'z': z,
        'Event Time Numeric': event_time_numeric,
        'Event Time Label': event_dates_str,
        'Event Time': event_time_dt
    })


    # Main scatter plot with linear color mapping
    fig = px.scatter_3d(
        df,
        x='x', y='y', z='z',
        color='Event Time Numeric',  # linear numeric scale
        color_continuous_scale='Viridis',
        title=plot_title,
        labels={
            'X': 'X []',
            'Y': 'Y []',
            'Z': 'Z []',
            'Event Time Numeric': 'Event Time'
        },
        hover_name='Event Time Label',
        hover_data={'Event Time Numeric': False},
        width=800,  # Set figure width
        height=600  # Set figure height
    )

    fig.update_traces(marker=dict(size=4, opacity=0.4))

    # Earth marker (larger, blue)
    fig.add_trace(go.Scatter3d(
        x=[0], y=[0], z=[0],
        mode='markers+text',
        marker=dict(size=10, color='blue', line=dict(color='black', width=2)),
        text=['Earth'],
        textposition='top center',
        name='Earth',
        hoverinfo='text'
    ))

    # Manually set colorbar ticks to formatted dates
    tick_vals = np.linspace(event_time_numeric.min(), event_time_numeric.max(), 5)
    tick_text = [pd.to_datetime(int(val), origin='unix', unit='D').strftime('%Y-%m') for val in tick_vals]

    fig.update_layout(coloraxis_colorbar=dict(
        title='Event Time',
        tickvals=tick_vals,
        ticktext=tick_text
    ))

    # L1 marker (smaller, red)
    fig.add_trace(go.Scatter3d(
        x=[238], y=[0], z=[0],
        mode='markers+text',
        marker=dict(size=5, color='red', line=dict(color='black', width=1)),
        text=['L1'],
        textposition='top center',
        name='L1',
        hoverinfo='text'
    ))

    # Customize layout
    fig.update_traces(marker=dict(opacity=0.8))
    fig.update_layout(
        scene=dict(
            xaxis=dict(title='X: [Earth radii]', range=[np.min(x), np.max(x)]),
            yaxis=dict(title='Y: [Earth radii]', range=[np.min(y), np.max(y)]),
            zaxis=dict(title='Z: [Earth radii]', range=[np.min(z), np.max(z)])
        ),
        legend=dict(x=0.8, y=0.9),
        margin=dict(l=0, r=0, b=0, t=50)
    )

    fig.show()

In [4]:
GSE_sc_plot(df_Wind, 'ICME: Wind Spacecraft in GSE Coordinates')